In [18]:
#load ml_modeling_module 
from ml_modeling_module import *
from os import walk

#first let's plot sige bandstructure
#read QE output file with bands for 0% strain
data=read_data('bands/si2ge2_0.dat')
bands0=data.read_bands_ml(fermi=6.18, bands_num=4)

#read QE output file with bands for 5% strain
data=read_data('bands/si2ge2_5.dat')
bands5=data.read_bands_ml(fermi=5.60, bands_num=4)
# reshape data: 8 bands 11 k-points
bands0=bands0.reshape(8,11)
bands5=bands5.reshape(8,11)

import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2,subplot_titles=("strain 0%", "strain 5%"))

for i in range(bands0.shape[0]):
    fig.add_trace(go.Scatter(x=np.arange(1,12,1), y=bands0[i],
                        mode='lines', marker=dict(color='black'), showlegend=False, 
                        name='markers'), col=1, row=1)  
for i in range(bands5.shape[0]):
    fig.add_trace(go.Scatter(x=np.arange(1,12,1), y=bands5[i],
                        mode='lines', marker=dict(color='black'), showlegend=False, 
                        name='markers'), col=2, row=1) 
fig.add_trace(go.Scatter(x=[1,11], y=[0,0],
                        mode='lines', line=dict(color= "RoyalBlue",width=3, dash='dot'), showlegend=False, 
                        name='markers'), col=1, row=1) 
fig.add_trace(go.Scatter(x=[1,11], y=[0,0],
                        mode='lines', line=dict(color= "RoyalBlue",width=3, dash='dot'), showlegend=False, 
                        name='markers'), col=2, row=1) 
fig.update_xaxes(showgrid=True,  
                 gridwidth=0.5, 
                 gridcolor='lightgrey', 
                 ticks='inside' ,                        
                 linecolor= 'black',                      
                 linewidth= 1,
                 mirror= True,
                 ticktext=['G', 'Z'],
                 tickvals=[1,11],
                 range=[1,11],
                 )
fig.update_yaxes(title='Energy, eV', 
                 showgrid=True,  
                 gridwidth=0.5, 
                 gridcolor='lightgrey', 
                 ticks='inside' ,                        
                 linecolor= 'black',
                 linewidth= 1,
                 mirror= True,
                )

fig.update_layout(width=900, height=650, plot_bgcolor='white')
fig.show()



In [10]:
# reading cell parameters from QE output file : NEED TO add a fucntion 
#(lattice constants are in bohr)
# X=np.array([[10.34568, 10.79211],
#             [10.45682, 10.71265],
#             [10.56082, 10.64615], 
#             [10.66056, 10.57623],
#             [10.76056, 10.50342],
#             [10.87056, 10.42435],
#             [10.60789, 10.610219]
#            ])
# X.shape


mypath='cell_param/'
filenames = next(walk(mypath), (None, None, []))[2]

lattice=[]
fermi=[]
for file in filenames:
    data=read_data(mypath+file)
    (a, c, composition) = data.read_lattice_parameters()
    lattice.append ([a, c, composition])
    
    fermi_level = data.read_fermi_levle()
    fermi.append(fermi_level)
    
X=lattice=np.array(lattice)



In [11]:
# reading bands from all QE output file

mypath='bands/'
filenames = next(walk(mypath), (None, None, []))[2]

y=[]
for i in range(len(filenames)):
    data=read_data(mypath+filenames[i])
    bands_ml=data.read_bands_ml(fermi=fermi[i], bands_num=2)
    y= np.append(y, bands_ml)
y=y.reshape(len(filenames),44)
y.shape

(7, 44)

# prepare data for modeling: split for training and testig sets

In [12]:
my_data=data_preparation(X,y)
(X_train, X_test, Y_train, Y_test)=my_data.select_train_test(test_row_number=[6])
#random split could also be used
#(X_train, X_test, Y_train, Y_test)=my_data.random_split(train_split=0.8, seed=42)

my_model=modeling(X_train, Y_train, X_test, Y_test, scaler='None')

#train Random forest model with No initial guess
(y_fit_RF, y_predict_RF, trials_RF, best_space_RF, regressorRF, text_RF)=my_model.RandomForest( max_evals=10, timeout=150*60,loss_threshold=0.01, cv=5, verbose=1)

print(text_RF)


#

Time taken for RF: 0:00:05.295193
Train:
R^2:0.91 RMSE:0.03
Test:
R^2:nan RMSE: 0.04


C:\Users\exx\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').

C:\Users\exx\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_regression.py:582: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.



In [7]:
## DOES NOT WORK YET 

# train XGboost model with initial guess
# best_space = {'alpha': 0.0, 'max_depth': 3.0, 'min_split_loss': 5.6, 'n_estimators': 100, 'x_reg_lambda': 1.00, 'x_subsample': 1.0}
# (y_fit_XG, y_predict_XG, trials_XG, best_space_XG, regressorXGB, text_XG)=my_model.XGB( max_evals=4, timeout=150*60,loss_threshold=0.01,init_vals=[best_space],cv=5, verbose=1) 
# print(text_XG)

C:\Users\exx\Anaconda3\envs\tensorflow\lib\site-packages\xgboost\core.py:614: UserWarning:

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption



Time taken for XGB: 0:07:34.327627
Train:
R^2:-0.0 RMSE:0.11
Test:
R^2:nan RMSE: 0.08


C:\Users\exx\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_regression.py:582: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.



# let's plot predicted bands vs calculated bands

In [13]:
# let's plot predicted bands vs calculated bands
#reading calculated bands:
data=read_data('bands/si2ge2_relax.dat')
bands=data.read_bands_ml(fermi=5.75, bands_num=4)
bands=bands.reshape(8,11)
#predicted bands
bands_RF=y_predict_RF.reshape(4,11)
#bands_XG=y_predict_XG.reshape(4,11)

import plotly.graph_objects as go
fig = go.Figure()

for i in range(bands.shape[0]):
    fig.add_trace(go.Scatter(x=np.arange(1,12,1), y=bands[i],
                        mode='lines', marker=dict(color='black'), showlegend=False, 
                        name='markers'))
    
for i in range(bands_RF.shape[0]):
    fig.add_trace(go.Scatter(x=np.arange(1,12,1), y=bands_RF[i],
                        mode='markers', marker=dict(color='red'), showlegend=False, 
                        name='markers'))
fig.add_trace(go.Scatter(x=[1,11], y=[0,0],
                        mode='lines', line=dict(color= "RoyalBlue",width=3, dash='dot'), showlegend=False, 
                        name='markers'))    

fig.update_xaxes(showgrid=True,  
                 gridwidth=0.5, 
                 gridcolor='lightgrey', 
                 ticks='inside' ,                        
                 linecolor= 'black',                      
                 linewidth= 1,
                 mirror= True,
                 ticktext=['G', 'Z'],
                 tickvals=[1,11],
                 range=[1,11]
                 )
fig.update_yaxes(title='Energy, eV', 
                 showgrid=True,  
                 gridwidth=0.5, 
                 gridcolor='lightgrey', 
                 ticks='inside' ,                        
                 linecolor= 'black',
                 linewidth= 1,
                 mirror= True,
                )
fig.update_layout(title='Si2Ge2 relaxed', width=450, height=650, plot_bgcolor='white')
fig.show()

# example of solving reverse problem 

In [17]:
# in this example we predict lattice constants for a given bandstructure (lattice constants are in bohr)

import GPyOpt
from GPyOpt.methods import BayesianOptimization
from sklearn.metrics import mean_squared_error

data=read_data('bands/si2ge2_relax.dat')
bands=data.read_bands_ml(fermi=5.75, bands_num=2)


def objfunc(x):

    x1 = float(x[0,0])
    x2 = float(x[0,1])
    x3 = float(x[0,2])

    return  mean_squared_error(regressorRF.predict([[x1,x2,x3]]).reshape(1,44), bands.reshape(1,44))

maxiter = 40
domain =    [
             {'name': 'a_paral', 'type': 'continuous', 'domain': (10, 11)},
             {'name': 'a_perp',  'type': 'continuous', 'domain': (10, 11)},
             {'name': 'composition',  'type': 'continuous', 'domain': (0, 1)}
            ]



Bopt = GPyOpt.methods.BayesianOptimization(objfunc, domain=domain)
Bopt.run_optimization(max_iter = maxiter)

print("Lattice parameters:"+str(Bopt.x_opt))    
print("MSE: "+str(Bopt.fx_opt)) 

Lattice parameters:[10.40470426 10.59502318  0.        ]
MSE: 0.002944861666935311
